In [1]:
#@title mount to drive (ドライブにマウント)
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Mounted at /content/drive
Tesla T4, 15360 MiB, 15101 MiB


In [2]:
# パッケージのインストール
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framewor

In [3]:
# パッケージのインポート
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
#@title Read dataset (データセットを読む)

#@markdown json_name
dataset_json_name = "mafuyu.json" #@param{type:"string"}

#@markdown result_name
result_name = "mafuyu" #@param{type:"string"}


import json
import os
from datasets import load_dataset, load_metric, Dataset



# 仮定: dataは指定されたJSONデータを含む
with open(os.path.join("/content/drive/MyDrive/Rinna",dataset_json_name),encoding = "utf-8") as f:
   yaoyao_data = json.load(f)

# text列を追加する関数
def add_text(entry):
    entry["text"] = f"""指示:
{entry["instruction"]}

応答:
{entry["output"]}"""
    return entry

# 各エントリに対してtext列を追加
updated_data = [add_text(entry) for entry in yaoyao_data]

text_data = [{"text": entry["text"]} for entry in updated_data]

# 新しいデータをDatasetオブジェクトに変換
dataset = Dataset.from_dict({"text": [entry["text"] for entry in text_data]})

In [17]:
print(dataset)
print(dataset[1]["text"])

Dataset({
    features: ['text'],
    num_rows: 157
})
指示:
まふゆは、どう思った？

応答:
・・・暖かくて、いいと思った...


In [18]:
# 量子化パラメータ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bitベースモデルの有効化
    bnb_4bit_quant_type="nf4",  # 量子化種別 (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 4bitベースモデルのdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 4bitベースモデルのネストされた量子化の有効化 (二重量子化)
)

# モデルの準備
model_name = "rinna/japanese-gpt-neox-3.6b-instruction-sft"
model = AutoModelForCausalLM.from_pretrained(
    model_name,  # モデル名
    quantization_config=bnb_config,  # 量子化パラメータ
    device_map={"": 0}  # モデル全体をGPU0にロード
)
model.config.use_cache = False  # キャッシュ (学習時はFalse)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True
)

In [19]:
#@title Train (学習)
# LoRAパラメータ
peft_config = LoraConfig(
    r=64,  # LoRAアテンションの次元
    lora_alpha=16,  # LoRAスケーリングのAlphaパラメータ
    lora_dropout=0.1,  # LoRA レイヤーのドロップアウト確率
    bias="none",  # LoRAのバイアス種別 ("none","all", "lora_only")
    task_type="CAUSAL_LM",  # タスク種別
    target_modules=["dense_4h_to_h", "dense", "dense_h_to_4h", "query_key_value"]
)


out_dir = "./result_"+str(result_name)
# 学習パラメータ
training_arguments = TrainingArguments(
    output_dir=out_dir,  # 出力ディレクトリ
    fp16=True,  # fp16学習の有効化 (T4:True,A100:False)
    bf16=False,  # bf16学習の有効化 (T4:False,A100:True)
    max_steps=800,  # 学習ステップ数
    per_device_train_batch_size=8,  # 学習用のGPUあたりのバッチサイズ
    gradient_accumulation_steps=1,  # 勾配を蓄積するための更新ステップの数
    optim="paged_adamw_32bit",  # オプティマイザ
    learning_rate=2e-4,  # 初期学習率 (AdamW オプティマイザー)
    lr_scheduler_type="cosine",  # 学習率スケジュール
    max_grad_norm=0.3,  # 最大法線勾配 (勾配クリッピング)
    warmup_ratio=0.03,  # 線形ウォームアップのステップ比率 (0から学習率まで)
    weight_decay=0.001,  # bias/LayerNormウェイトを除く全レイヤーに適用するウェイト減衰
    save_steps=0,  # 何ステップ毎にチェックポイントを保存するか
    logging_steps=25,  # 何ステップ毎にログを記録するか
    group_by_length=True,  # シーケンスを同じ長さのバッチにグループ化 (メモリ節約して学習速度が大幅アップ)
    report_to="tensorboard"  # レポート
)

# SFTパラメータ
trainer = SFTTrainer(
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=dataset,  # データセット
    dataset_text_field="text",  # データセットのtext列
    peft_config=peft_config,  # PEFTパラメータ
    args=training_arguments,  # 学習パラメータ
)

# モデルの学習
trainer.train()
trainer.model.save_pretrained(os.path.join("/content/drive/MyDrive/Rinna",out_dir))


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/157 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.603700
50,2.390700
75,1.761400
100,0.973700
125,0.463800
150,0.300600
175,0.246300
200,0.225600
225,0.205600
250,0.202400


In [20]:
#@title 学習終わりの推論

#@markdown json_name
prompt = "\u8CB4\u65B9\u306E\u540D\u524D\u306F\uFF1F" #@param{type:"string"}


final_prompt = f"""指示:{prompt}

応答:
"""

# 推論の実行
input_ids = tokenizer.encode(final_prompt, add_special_tokens=False, return_tensors="pt")
output_ids = model.generate(
    input_ids.to(device=model.device),
    max_length=200,
    temperature=0.7,
    do_sample=True,
)
output = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):])

print("")
print(prompt)
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



貴方の名前は？
・・・朝比奈まふゆ</s>


In [21]:
#@title 推論のみ
# モデルの準備
import os
model_name = "rinna/japanese-gpt-neox-3.6b-instruction-sft"

out_dir = "./result_"+str(result_name)
print(out_dir)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bitベースモデルの有効化
    bnb_4bit_quant_type="nf4",  # 量子化種別 (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 4bitベースモデルのdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 4bitベースモデルのネストされた量子化の有効化 (二重量子化)
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = PeftModel.from_pretrained(
    base_model,
    os.path.join("/content/drive/MyDrive/Rinna",out_dir)
)

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    add_eos_token=True,
    trust_remote_code=True
)

./result_mafuyu


In [22]:
# プロンプトの準備

#@markdown json_name
prompt = "\u8CB4\u65B9\u306E\u540D\u524D\u306F\uFF1F" #@param{type:"string"}


final_prompt = f"""指示:\n{prompt}

応答:
"""

# 推論の実行
input_ids = tokenizer.encode(final_prompt, add_special_tokens=False, return_tensors="pt")
output_ids = model.generate(
    input_ids=input_ids.to(device=model.device),
    max_length=200,
    temperature=0.7,
    do_sample=True,
)
output = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):])

print("")
print(prompt)
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.



貴方の名前は？
・・・朝比奈まふゆ</s>
